# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np


In [2]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [3]:
# frequency table for prestige and whether or not someone was admitted
print pd.crosstab(df['admit'], df['prestige'], rownames=['admit'])

prestige  1.0  2.0  3.0  4.0
admit                       
0          28   95   93   55
1          33   53   28   12


In [4]:
df.groupby("prestige").count()

,admit,gre,gpa
prestige,,,
1.0,61,61,61
2.0,148,148,148
3.0,121,121,121
4.0,67,67,67


In [5]:
df["prestige"]= df.prestige.astype('int64')

/Users/edmond_20000/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [6]:
dummy_ranks = pd.get_dummies(df['prestige'], prefix='prestige')

In [7]:
prestige_dummy = pd.get_dummies(df.prestige, prefix = "prestige", drop_first= True) 
prestige_dummy.head()


,prestige_2,prestige_3,prestige_4
0,0,1,0
1,0,1,0
2,0,0,0
3,0,0,1
4,0,0,1


In [8]:
# prestige_dummy = pd.get_dummies(df.prestige)
# prestige_dummy.head()

In [9]:
df.head()

,admit,gre,gpa,prestige
0,0,380.0,3.61,3
1,1,660.0,3.67,3
2,1,800.0,4.00,1
3,1,640.0,3.19,4
4,0,520.0,2.93,4


In [10]:
newdf = df.join(prestige_dummy)

In [11]:
newdf.head()

,admit,gre,gpa,prestige,prestige_2,prestige_3,prestige_4
0,0,380.0,3.61,3,0,1,0
1,1,660.0,3.67,3,0,1,0
2,1,800.0,4.00,1,0,0,0
3,1,640.0,3.19,4,0,0,1
4,0,520.0,2.93,4,0,0,1


#### 2.2 When modeling our class variables, how many do we need? 



Answer: 3

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [12]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_1':])
print handCalc.head()

   admit    gre   gpa  prestige_1  prestige_2  prestige_3  prestige_4
0      0  380.0  3.61           0           0           1           0
1      1  660.0  3.67           0           0           1           0
2      1  800.0  4.00           1           0           0           0
3      1  640.0  3.19           0           0           0           1
4      0  520.0  2.93           0           0           0           1


In [13]:
admit_prestige = pd.crosstab(index = df.admit, columns = df.prestige, margins = True)
admit_prestige.index = ["denied", "admitted", "Total"]
admit_prestige

prestige,1,2,3,4,All
denied,28,95,93,55,271
admitted,33,53,28,12,126
Total,61,148,121,67,397


In [14]:
df.groupby("admit").count()

,gre,gpa,prestige
admit,,,
0,271,271,271
1,126,126,126


In [15]:
df.groupby("gre").count()

,admit,gpa,prestige
gre,,,
220.0,1,1,1
300.0,3,3,3
340.0,4,4,4
360.0,4,4,4
380.0,8,8,8
400.0,11,11,11
420.0,7,7,7
440.0,10,10,10
460.0,13,13,13


In [16]:
df.groupby("gpa").count()

,admit,gre,prestige
gpa,,,
2.26,1,1,1
2.42,2,2,2
2.48,1,1,1
2.52,1,1,1
2.55,1,1,1
2.56,1,1,1
2.62,2,2,2
2.63,1,1,1
2.65,1,1,1


In [17]:
admit_prestige1 = pd.crosstab(index = handCalc.admit, columns = handCalc["prestige_1"], margins = True)
admit_prestige1.index = ["denied", "admitted", "Total"]
admit_prestige1.columns = ["none", "one", "All"]
admit_prestige1 
#crosstab prestige admission and checking to see if any student was accepted

,none,one,All
denied,243,28,271
admitted,93,33,126
Total,336,61,397


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [18]:
#finding the probability of being accepted
print "Number accepted with prestige of 1: {}".format(admit_prestige1[admit_prestige1.index == "admitted"].one)
print "Total number with prestige of 1: {}".format(admit_prestige1[admit_prestige1.index == "Total"].one)

Number accepted with prestige of 1: admitted    33
Name: one, dtype: int64
Total number with prestige of 1: Total    61
Name: one, dtype: int64


In [19]:
(admit_prestige1[admit_prestige1.index == "admitted"].one)/float(admit_prestige1[admit_prestige1.index == "Total"].one)

admitted    0.540984
Name: one, dtype: float64

In [20]:
print "Students attending #1 ranked college have an {}% rate".format(
    (admit_prestige1[admit_prestige1.index == "admitted"].one)/float(admit_prestige1[admit_prestige1.index == "Total"].one)*100)

Students attending #1 ranked college have an admitted    54.098361
Name: one, dtype: float64% rate


In [21]:
#odds of admission acceptance when a student attends a #1 ranked college
Odds1 = (admit_prestige1[admit_prestige1.index == "admitted"].one)/float(admit_prestige1[admit_prestige1.index == "denied"].one)
Odds1

admitted    1.178571
Name: one, dtype: float64

#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [22]:
print "Accepted with prestige of 1: {}".format(admit_prestige1[admit_prestige1.index == "admitted"].none)
print "Total accepted with prestige of 1: {}".format(admit_prestige1[admit_prestige1.index == "Total"].none)

Accepted with prestige of 1: admitted    93
Name: none, dtype: int64
Total accepted with prestige of 1: Total    336
Name: none, dtype: int64


In [23]:
OddsNone = (admit_prestige1[admit_prestige1.index == "admitted"].none)/float(admit_prestige1[admit_prestige1.index == "denied"].none)
OddsNone

admitted    0.382716
Name: none, dtype: float64

In [24]:
#Total odds of admission if student did not attend a #1 ranked college
OddsNone = (admit_prestige1[admit_prestige1.index == "admitted"].none)/float(admit_prestige1[admit_prestige1.index == "denied"].none)
OddsNone

admitted    0.382716
Name: none, dtype: float64

#### 3.3 Calculate the odds ratio

In [25]:
#calculating the odds ratio. With odds divde by without odds
OR = float(Odds1) / OddsNone
OR

admitted    3.079493
Name: none, dtype: float64

#### 3.4 Write this finding in a sentenance: 

Answer: This seems to infer that a student is 3 times more likely to be admitted to university if the student attended higher ranked prestigious school than a lower ranked school.

#### 3.5 Print the cross tab for prestige_4

In [26]:
#crosstab prestige admission and checking to see if any student was accepted
admit_prestige4 = pd.crosstab(index = handCalc.admit, columns = handCalc["prestige_4"], margins = True)
admit_prestige4.index = ["denied", "admittted", "Total"]
admit_prestige4.columns = ["not4", "four", "All"]
admit_prestige4

,not4,four,All
denied,216,55,271
admittted,114,12,126
Total,330,67,397


#### 3.6 Calculate the OR 

In [62]:
Odds4 = admit_prestige4[admit_prestige4.index == "admitted"].four/float(
    admit_prestige4[admit_prestige4.index == "denied"].four)

print Odds4

OddsNot4 = admit_prestige4[admit_prestige4.index == "admitted"].not4/float(
    admit_prestige4[admit_prestige4.index == "denied"].not4)

print OddsNot4

print "Odds Ratio for lowest prestige college {} to 1".format(Odds4/OddsNot4)

Series([], Name: four, dtype: float64)
Series([], Name: not4, dtype: float64)
Odds Ratio for lowest prestige college Series([], dtype: float64) to 1


#### 3.7 Write this finding in a sentence

Answer:

## Part 4. Analysis

In [28]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(prestige_dummy.ix[:, 'prestige_2.0':])
data.head()

,admit,gre,gpa,prestige_3,prestige_4
0,0,380.0,3.61,1,0
1,1,660.0,3.67,1,0
2,1,800.0,4.00,0,0
3,1,640.0,3.19,0,1
4,0,520.0,2.93,0,1


In [29]:
df = pd.read_csv("../assets/admissions.csv")
df.head()

,admit,gre,gpa,prestige
0,0,380.0,3.61,3.0
1,1,660.0,3.67,3.0
2,1,800.0,4.00,1.0
3,1,640.0,3.19,4.0
4,0,520.0,2.93,4.0


In [30]:
newdf.describe()

,admit,gre,gpa,prestige,prestige_2,prestige_3,prestige_4
count,397.000000,397.000000,397.000000,397.000000,397.000000,397.000000,397.000000
mean,0.317380,587.858942,3.392242,2.488665,0.372796,0.304786,0.168766
std,0.466044,115.717787,0.380208,0.947083,0.484159,0.460898,0.375017
min,0.000000,220.000000,2.260000,1.000000,0.000000,0.000000,0.000000
25%,0.000000,520.000000,3.130000,2.000000,0.000000,0.000000,0.000000
50%,0.000000,580.000000,3.400000,2.000000,0.000000,0.000000,0.000000
75%,1.000000,660.000000,3.670000,3.000000,1.000000,1.000000,0.000000
max,1.000000,800.000000,4.000000,4.000000,1.000000,1.000000,1.000000


In [31]:
newdf.dropna(inplace=True)

In [32]:
newdf.shape

(397, 7)

In [33]:
#http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
from sklearn.linear_model import LogisticRegression
lm = LogisticRegression()

In [34]:
X = newdf[['gre', 'gpa', 'prestige_2', 'prestige_3', 'prestige_4']]
y = newdf.admit

In [35]:
model  = lm.fit(X, y)

In [36]:
model.coef_

array([[ 0.00178497,  0.23229458, -0.60347467, -1.17214957, -1.37729795]])

We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [37]:
# manually add the intercept
data['intercept'] = 1.0
data.head()

,admit,gre,gpa,prestige_3,prestige_4,intercept
0,0,380.0,3.61,1,0,1.0
1,1,660.0,3.67,1,0,1.0
2,1,800.0,4.00,0,0,1.0
3,1,640.0,3.19,0,1,1.0
4,0,520.0,2.93,0,1,1.0


#### 4.1 Set the covariates to a variable called train_cols

In [38]:
#Looks like Kfold is the best option here looking at the link below
# http://scikit-learn.org/stable/modules/cross_validation.html
from sklearn.cross_validation import KFold

In [39]:
# using kfolds here so it will be x_train, y_train, x_test, y_test = x[train], x[test], y[train], y[test]
X = data
y = data.admit
kf = KFold(5, n_folds=3)
# kf = KFold(n_splits=2)
# kf.get_n_splits(X)

In [40]:
train_cols = ['gre', 'gpa', 'prestige_3', 'prestige_4', 'intercept']

#### 4.2 Fit the model

In [41]:
newdf.head()

,admit,gre,gpa,prestige,prestige_2,prestige_3,prestige_4
0,0,380.0,3.61,3,0,1,0
1,1,660.0,3.67,3,0,1,0
2,1,800.0,4.00,1,0,0,0
3,1,640.0,3.19,4,0,0,1
4,0,520.0,2.93,4,0,0,1


In [42]:
# stats models Logit function
logistic = sm.Logit (data['admit'], data[train_cols])
result = logistic.fit()

Optimization terminated successfully.
         Current function value: 0.579685
         Iterations 6


In [43]:
newdf["intercept"] = 0

In [44]:
newdf.head()

,admit,gre,gpa,prestige,prestige_2,prestige_3,prestige_4,intercept
0,0,380.0,3.61,3,0,1,0,0
1,1,660.0,3.67,3,0,1,0,0
2,1,800.0,4.00,1,0,0,0,0
3,1,640.0,3.19,4,0,0,1,0
4,0,520.0,2.93,4,0,0,1,0


In [45]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      392
Method:                           MLE   Df Model:                            4
Date:                Thu, 12 Jan 2017   Pseudo R-squ.:                 0.07232
Time:                        21:33:30   Log-Likelihood:                -230.14
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 3.057e-07
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
gre            0.0022      0.001      2.064      0.039         0.000     0.004
gpa            0.8188      0.330      2.481      0.013         0.172     1.466
prestige_3    -0.8666      0.266     -3.262      0.001        -1.387    -0.346
prestige_4    -1.0761      0.354     -3.041      0.002        -1.770    -0.382
intercept     -4.5016      1.107     -4.065      0.000        -6.672    -2.331
==============================================================================
"""

#### 4.3 Print the summary results

In [46]:
print result.summary()

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      392
Method:                           MLE   Df Model:                            4
Date:                Thu, 12 Jan 2017   Pseudo R-squ.:                 0.07232
Time:                        21:33:30   Log-Likelihood:                -230.14
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 3.057e-07
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
gre            0.0022      0.001      2.064      0.039         0.000     0.004
gpa            0.8188      0.330      2.481      0.013         0.172     1.466
prestige_3    -0.8666      0.266     -3.262      0.0

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [47]:
# first the exponential of each of the coefficients
expCoeff = np.exp(result.params)
expCoeff

gre           1.002244
gpa           2.267721
prestige_3    0.420395
prestige_4    0.340913
intercept     0.011092
dtype: float64

In [48]:
#interval - confidence
confInt = result.conf_int()
confInt

,0,1
gre,0.000113,0.004370
gpa,0.172041,1.465510
prestige_3,-1.387205,-0.345918
prestige_4,-1.769800,-0.382456
intercept,-6.672056,-2.331092


In [49]:
confInt['OR'] = result.params
confInt

,0,1,OR
gre,0.000113,0.004370,0.002241
gpa,0.172041,1.465510,0.818775
prestige_3,-1.387205,-0.345918,-0.866562
prestige_4,-1.769800,-0.382456,-1.076128
intercept,-6.672056,-2.331092,-4.501574


#### 4.5 Interpret the OR of Prestige_2

Answer: GPA increases with each unit change

#### 4.6 Interpret the OR of GPA

Answer: 

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [50]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [51]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print 'GRE: {}'.format(gres)
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print 'GPA: {}'.format(gpas)
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

GRE: [ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
GPA: [ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


In [52]:
combos.head()

,0,1,2,3
0,220.0,2.260000,1.0,1.0
1,220.0,2.260000,2.0,1.0
2,220.0,2.260000,3.0,1.0
3,220.0,2.260000,4.0,1.0
4,220.0,2.453333,1.0,1.0


In [53]:
combos.groupby(0).count()

,1,2,3
0,,,
220.000000,40,40,40
284.444444,40,40,40
348.888889,40,40,40
413.333333,40,40,40
477.777778,40,40,40
542.222222,40,40,40
606.666667,40,40,40
671.111111,40,40,40
735.555556,40,40,40


In [54]:
combos.groupby(1).count()

,0,2,3
1,,,
2.260000,40,40,40
2.453333,40,40,40
2.646667,40,40,40
2.840000,40,40,40
3.033333,40,40,40
3.226667,40,40,40
3.420000,40,40,40
3.613333,40,40,40
3.806667,40,40,40


In [55]:
combos.groupby(2).count()

,0,1,3
2,,,
1.0,100,100,100
2.0,100,100,100
3.0,100,100,100
4.0,100,100,100


#### 5.1 Recreate the dummy variables

In [56]:
# recreate the dummy variables
combos.columns = ['gre', 'gpa', 'prestige', 'intercept']
dummy_ranks = pd.get_dummies(combos['prestige'], prefix='prestige')
dummy_ranks.columns = ['prestige_1.0', 'prestige_2.0', 'prestige_3.0', 'prestige_4.0']

# keep only what we need for making predictions
cols_to_keep = ['gre', 'gpa', 'prestige', 'intercept']
combos = combos[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2.0':])
combos.head()

,gre,gpa,prestige,intercept,prestige_2.0,prestige_3.0,prestige_4.0
0,220.0,2.260000,1.0,1.0,0,0,0
1,220.0,2.260000,2.0,1.0,1,0,0
2,220.0,2.260000,3.0,1.0,0,1,0
3,220.0,2.260000,4.0,1.0,0,0,1
4,220.0,2.453333,1.0,1.0,0,0,0


#### 5.2 Make predictions on the enumerated dataset

In [57]:
#not sure if this is correct but 
combos['prediction'] = result.predict(combos[train_cols])
combos.head ()

KeyError: "['prestige_3' 'prestige_4'] not in index"

In [58]:
combos.tail(4)

,gre,gpa,prestige,intercept,prestige_2.0,prestige_3.0,prestige_4.0
396,800.0,4.0,1.0,1.0,0,0,0
397,800.0,4.0,2.0,1.0,1,0,0
398,800.0,4.0,3.0,1.0,0,1,0
399,800.0,4.0,4.0,1.0,0,0,1


#### 5.3 Interpret findings for the last 4 observations

Answer: Can infer here that acceptance varies according to the prestige even with the same gpa and gre. 

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.